In [19]:
from PIL import Image
from io import BytesIO
from models.load_model import load_torch_model
from models.model_wrappers import SegmentationModelAI, ImageType, OnnxSegmentationWrapper
from models.utils import ImageToVector, load_dummy_image, torch_to_onnx
from skimage.transform import resize
import numpy as np

from models.utils import segment_prediction_to_image, vector_to_pil
# Loading the model
model, preprocessor = load_torch_model()
dummy_batch = preprocessor(load_dummy_image()).unsqueeze(0)

# Converting it to an onnx model
model = torch_to_onnx(model, dummy_batch)
# preprocessor = torch_to_onnx(preprocessor, dummy_batch)
# setting up the wrapper
# model = SegmentationModelAI(model, preprocessor)

In [5]:
# preprocessor.get_inputs()[0].name

'img'

In [6]:
# preprocessor.get_outputs()[0].name

'4'

In [11]:
from models.model_wrappers import ModelWrapper, InferenceSession, Optional, Callable
class OnnxWrapper(ModelWrapper):

    def __init__(self, model: InferenceSession, preprocessor: Optional[Callable] = None):
        super().__init__(model, preprocessor)
        self.input_names = [x.name for x in self.model.get_inputs()]
        self.output_names = [x.name for x in self.model.get_outputs()]


    @property
    def supported_model_types(self):
        return (InferenceSession,)

    def predict_batch(self, images: np.array) -> np.array:
        """Prediction of a batch

        Args:
            images (np.array): batch of images in shape [N, 3, H, W]

        Returns:
            np.array: the result of the model
        """
        if self.preprocessor:
            images = self.preprocessor(images)
        model_input = {self.model.get_inputs()[0].name: images}
        return self.model.run(self.output_names, model_input)[0]
    
    @staticmethod
    def permute_image(image: np.array) -> np.array:
        """
        Permute image of [H, W, 3] to [3, H, W] if needed
        """
        if image.shape[0] != 3 and image.shape[-1] == 3:
            image = np.transpose(image, (2, 1, 0))
        return image
    
    def predict_single(self, image: ImageType) -> np.array:
        """Prediction of a single data point (unbatched)

        Args:
            image (ImageType): The input image (unbatched) of shape [3, H, W]

        Returns:
            np.array: The prediction of the model
        """
        image = ImageToVector.to_numpy(image)
        image = self.permute_image(image)
        # Expand dimmentions
        image = image[np.newaxis, :]
        return self.predict_batch(image)[0]


In [23]:
w = OnnxWrapper(torch_to_onnx(preprocessor, dummy_batch))

<onnxruntime.capi.onnxruntime_inference_collection.InferenceSession object at 0x12cb705f0> <class 'onnxruntime.capi.onnxruntime_inference_collection.InferenceSession'>


In [34]:
i = load_dummy_image()
i = np.array(i, dtype=np.float32)[np.newaxis, :]
# preprocessor(i)
w.predict_batch(i)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: img for the following indices
 index: 1 Got: 1026 Expected: 3
 index: 2 Got: 1282 Expected: 520
 index: 3 Got: 3 Expected: 649
 Please fix either the inputs/outputs or the model.

(1, 1026, 1282, 3)